In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from collections import namedtuple
#from pyspark.streaming import StreamingContext

## Setup Spark Streaming

In [2]:
spark = SparkSession.builder.appName('twitter_structured_streaming').getOrCreate()
#sc = spark.sparkContext
#ssc = StreamingContext(sc, 10)

In [3]:
#socket_stream = ssc.socketTextStream("0.0.0.0", 5555)

In [4]:
#time_window = 10
#lines = socket_stream.window(time_window)
lines = spark.readStream \
    .format("socket") \
    .option("host", '0.0.0.0') \
    .option("port", 5555) \
    .load()

In [5]:
#fields = ("tag","count")
#Tweet = namedtuple('Tweet',fields)

In [6]:
sep = ',|,lksd\\|.k'

split_cols = split(lines['value'],sep)
hashtags = lines.withColumn('created_at',split_cols.getItem(0)).withColumn('user_name',split_cols.getItem(1)) \
    .withColumn('hashtag',split_cols.getItem(2)).drop('value')

In [7]:
htCounts = hashtags.groupBy("hashtag").count()

In [7]:
lines_c = lines.groupBy('value').count()

In [8]:
query = htCounts.writeStream.queryName('count').outputMode('complete').format('console').start()
query.awaitTermination()

KeyboardInterrupt: 

In [36]:
query.stop()

In [9]:
spark.sql("select * from count").toPandas()

AnalysisException: 'Table or view not found: count; line 1 pos 14'

In [57]:
htCounts

DataFrame[hashtag: string, count: bigint]